#  05 GPT-2 Model Text Generation With gpt-2-simple (Colab notebook)

*Note: This notebook is an edited version the original Google Colab notebook by [Max Woolf](http://minimaxir.com), creator of the `gpt-2-simple` wrapper for OpenAI's GPT-2. Since this notebook provided an easy workflow directly from the creator as well as Colab's access to a free GPU, I decided to maximize efficiency and usability by using it as the base rather than building a new one.*

For more about `gpt-2-simple`, you can visit [Max's GitHub repository](https://github.com/minimaxir/gpt-2-simple). You can also read his [blog post](https://minimaxir.com/2019/09/howto-gpt2/) for a link to the original version and more information how to use this notebook.

### Imports

Please note: this wrapper uses TensorFlow v1.x

In [1]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## GPU
We'll verify which GPU is active by running the cell below.

In [2]:
!nvidia-smi

Wed Jan 27 10:36:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    24W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Mounting Google Drive

Next, we'll connect to our Google Drive by running the cell below. 

In [3]:
gpt2.mount_gdrive()

Mounted at /content/drive


## Load a Trained Model Checkpoint

Running the next cell will copy the `.tar` checkpoint file from your Google Drive into the Colaboratory VM. The filename will be in the form of `checkpoint_{run_name}.tar`.

In [4]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [5]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-10000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-10000


## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

**Some great notes from Max:**

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [6]:
def generate_examples(sess, length=150, temperature=0.7, 
                      top_p =0.0, top_k=0.0, seed = None, 
                      prefix='', nsamples=5, batch_size=5):
  result = gpt2.generate(sess,
                length=length,
                temperature=temperature,
                prefix=prefix,
                seed=seed,
                top_p = top_p,
                nsamples=nsamples,
                batch_size=batch_size
                )
  return result

In [7]:
# function to look at generative text given a passed list of temperatures
def generate_examples_by_temp(temperture_list, sess, prefix='', nsamples=1, batch_size=1):
  temperatures = temperature_list
  result = ''
  for temperature in temperatures:
    print(f'\nResults for temperature of {temperature}\n')
    generate_examples(sess=sess, temperature=temperature, prefix=prefix, nsamples=nsamples, batch_size=batch_size)
    print(f'\nEnd of Samples\n')
    print(f'*--*--*--*--*--*--*--*--*--*--*--*--*--*--')
  return 'END OF TEMPERATURE GENERATED EXAMPLES'

In [8]:
# temperature list to look at resultant outputs
temperature_list = [0.2, 0.5, 0.7, 1.0]

In [10]:
# setting the lead in prefix as an incomplete sample caption
sample_caption = 'A young couple stands on'

# generating generative examples by temperature for the sample caption
generate_examples_by_temp(temperature_list, sess, prefix=sample_caption)


Results for temperature of 0.2

A young couple stands on a balcony looking at a window. At
      their right hand is a table, and on their left a table,
      alternately of silk and papyrus. Here they areas well
      aware of the great beauty of their salon, but from the
      corner of the room they perceive a vast number of
      oriels, minarets, and pinnacles, allured by the rich
      draperies and costly lamps of the owner. Some few words
      are said between the tressels of the company, but the
      conversation is

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.5

A young couple stands on a balcony looking at a view of the river. At a corner of the
      canal steps steps lead into a side street. On the sidewalk is found a
      little building, bearing a large bell. It is opened and closed
      frequently. It is noticed that it is not much used at night.

      The ground of the street is irregularly shaped. It is a late
      a

'END OF TEMPERATURE GENERATED EXAMPLES'

In [11]:
# setting the lead in prefix as an incomplete sample caption
sample_caption = 'A wall of bricks'

# generating generative examples by temperature for the sample caption
generate_examples_by_temp(temperature_list, sess, prefix=sample_caption)


Results for temperature of 0.2

A wall of bricks
      and mortar, and a mass of loose bricks and mortar, all
       clustered into a very large and very impenetrable
       circle, with a long section of one of the bricks at the
       centre. On the edge of this circle, and a few paces from the
       centre, stood a human skeleton, posed evidently for the
       purpose of executing some sweeping movements for the
       safety of the surrounding world.

       But, as the great shakes and jumps in the boat were
    

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.5

A wall of bricks
      and mortar, together with a bed of oak, lay near us.

      We were nearly three hours in reaching the end of this passage,
      when we received a yell of fire. It was a prodigious sight to behold,
      even from the perspective of the savages. We scrambled to
      get up our weapons, but had no difficulty in doing so, when
      one of the men got th

'END OF TEMPERATURE GENERATED EXAMPLES'

In [13]:
# setting the lead in prefix as an incomplete sample caption
sample_caption = 'A man walks on the beach'

# generating generative examples by temperature for the sample caption
generate_examples_by_temp(temperature_list, sess, prefix=sample_caption)


Results for temperature of 0.2

A man walks on the beach.”

      “And the man who sat upon the rock,” said the king, “is the
      man who lies upon the sand.”

      “And the man who lies upon the rock,” said the king, “is the
      guardian of the shrine which is in the hills.”

      “And the man who lies upon the sand,” said the king, “is the
      son of Athelas.”

      “And the man who lies upon the rock,” said

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.5

A man walks on the beach.”

      “Man-of-the-World!”

      “What—what—what—what is it?”

      “I have no idea.”

      “Man-of-the-World!”

      “What—what—what is it?”

      “I have no idea either.”

      “Man-of-the-World!”

      “What—what—what is it?”

     

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.7

A man walks on the beach. A boat is seen in the distance. It is
      estimated that at least five or six of the natives

'END OF TEMPERATURE GENERATED EXAMPLES'

In [12]:
# setting the lead in prefix as an incomplete sample caption
sample_caption = 'A woman stands by a large'

# generating generative examples by temperature for the sample caption
generate_examples_by_temp(temperature_list, sess, prefix=sample_caption)


Results for temperature of 0.2

A woman stands by a large
      rock which leads into the recesses of the chasm. On the
      side of this rock is a hole which leads directly into the
      recesses of the morass, and forms the limit of the lowest
      and most accessible portion of the vale. Here the sun descends
      and disappears, and the morass is clothed in a misty and
      heavy fog.

      “The fogs and mist which encircle and enshrine the
      morass are the foul stalks of the ash trees which grow
 

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.5

A woman stands by a large window near the top of the building. On the
      mantelpieces are visible paintings in which are
      recognizable but untitled birds. Some paintings are
      represented as if they were part of the visual arts; and
      consequently convey ideas of artesian well-being. Others
      are simple landscapes composed with rough materials, such as
      green l

'END OF TEMPERATURE GENERATED EXAMPLES'

In [10]:
# setting the lead in prefix as an incomplete sample caption
sample_caption = 'The wind howls across the'

# generating generative examples by temperature for the sample caption
generate_examples_by_temp(temperature_list, sess, prefix=sample_caption)


Results for temperature of 0.2

The wind howls across the heaven,
      while the rain howls upon the earth—while the lightning
      howls upon the trees—and while the thunder howls upon the
      rocks—and while the lightning HOWLS upon the water-logged
                                                                                         

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.5

The wind howls across the heaven
      and the earth, and the stars are waxing lily-fringed.

      _P._ But where are the mimes?

      _V._ They are here!—here!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—mimes!—”

      _P._ And where are the nooks of the serpent?

      _V._ Here!—

End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.7

The wind howls across the heaven, and
      the stars sparkle radiantly; but men are chained within
      their prisons, and w

'END OF TEMPERATURE GENERATED EXAMPLES'

In [11]:
# setting the lead in prefix as an incomplete sample caption
sample_caption = 'The meaning of life is'

# generating generative examples by temperature for the sample caption
generate_examples_by_temp(temperature_list, sess, prefix=sample_caption)


Results for temperature of 0.2

The meaning of life is obscure. It is not the labor of the mind,
      to which God has subjected the merely human, but of the
      soul itself. It is the labor of the mind to know that
      man is a machine, to which God has subjected the
      immaterial, and to which God has no power, but to
      which God has made known the power of life.

      _P._ But you speak of _mind_ and of _consciousness_—pardon?

      _V._ Yes—to which God I have no access—but in regard to


End of Samples

*--*--*--*--*--*--*--*--*--*--*--*--*--*--

Results for temperature of 0.5

The meaning of life is obscure. It is not the labor of the mind,
      but the labor of the heart. It is the sympathy of the
      soul with its fellows—with its own kind. It is the fervid, the
      enduring, the perpetual love which blinds us to the
      exterior world. It is the love which exists within us,
      and animates our actions. It is the passion which
      animates our will—an

'END OF TEMPERATURE GENERATED EXAMPLES'

For bulk generation, `gpt-2-simple` allows to  generate a large amount of text. The next cell will generate a generated text file with a unique timestamp.

We can rerun the cells as many times as desired and then download the text files to our local computer.

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=250,
                      temperature=0.7,
                      prefix="\"My name is Poe. And you are?\"",
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### LICENSE
Since I'm extensively utilizing the framework and processes created by Max Woolf, please be aware of this licensing information related to said software.

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.